In [1]:
from tensorflow.examples.tutorials.mnist import input_data
import matplotlib.pyplot as plt
from numpy.linalg import norm
import numpy as np 
import time
import math

In [2]:
# 计时装饰器
def timer(func):
    def wrapper(*args, **kwargs):
        start_time = time.time()
        func(*args, **kwargs)
        end_time = time.time()
        print('Training time is :{:.2f} s.'.format(end_time - start_time))
    return wrapper

In [7]:
class WaveletNeuralNet(object):
    # 初始化神经网络，sizes是神经网络的层数和每层神经元个数  
    def __init__(self, sizes):
        self.sizes_ = sizes
        self.num_layers_ = len(sizes)  # 层数
        if self.num_layers_ > 3:
            print('ERROR!')
        self.num_nuerals_ = sizes[1]#隐含层个数 w为
        self.w_ = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]  # w_、b_初始化为正态分布随机数
        self.b_ = [np.random.randn(y, 1) for y in sizes[1:]]#numpy.random.randn(d0, d1, …, dn)是从标准正态分布中返回一个或多个样本值
        self.t_ = np.random.randint(2, 15, (self.num_nuerals_, 1))#np.random.randint(low,high,size))
#         self.t_ = np.random.normal(5, 2., (self.num_nuerals_, 1))
        self.s_ = 2 * np.random.randn(self.num_nuerals_, 1)
    
    #欧氏距离
    def oushi(self,v1,v2):
        v1=np.array(v1);v2=np.array(v2)
        op2=np.linalg.norm(v1-v2)
        return op2
    
    # Sigmoid函数，S型曲线，
    def sigmoid(self, z):
        return 1.0 / (1.0 + np.exp(-z))
    
    # Sigmoid函数的导函数
    def sigmoid_der(self, z):
        return self.sigmoid(z) * (1 - self.sigmoid(z))
    
    # morlet小波母函数
    def phi(self, z, t=1, s=0):
        z_ = (z - s) / t
        return np.cos(1.75 * z_) * np.exp(-z_**2 / 2.)
    
    # 小波函数导数
    def phi_der(self, z, t=1, s=0):
        z_ = (z - s) / t
        return (-1.75 * np.sin(1.75 * z_) * np.exp(-z_**2 / 2) - z_ * np.cos(1.75 * z_) * np.exp(-z_**2 / 2)) / t
    
    def feedforward(self, x): # 前向：各层网络节点计算
        n = self.w_[0].shape[1]#输入层个数
        x = (np.array(x)).reshape(n,-1)#将数据准换成一行
        x1 = self.phi(np.dot(self.w_[0], x) + self.b_[0], self.t_, self.s_)#np.dot点积 小波计算：输入层到输出层
        x2 =np.dot(self.w_[1], x1) + self.b_[1]#self.sigmoid(np.dot(self.w_[1], x1) + self.b_[1])#sigmoid计算：
        '''隐含层到输出层'''
        return x2
    
    # 反向传播
    def backprop(self, x, y):#反向传播更新参数
        b_new = [np.ones(b.shape) for b in self.b_]
        w_new = [np.ones(w.shape) for w in self.w_]
        t_new = self.t_
        s_new = self.s_
        activation = x
        
        activations = [x]  # activations代表着每层的输出
        #print('activation',np.array(activation))
        #print('activation',np.array(activation).size)
        zs = []  # zs记录隐含层与输出层的数据
        z = np.dot(self.w_[0],activation.reshape(-1,1))+ self.b_[0]#zl(1)
        #print('w0',np.array((self.w_[0])).shape)#print('z',np.array(z).shape)
        
        zs.append(z)#zl(1)
        activation = self.phi(z, t_new, s_new)## 计时装饰器

        activations.append(activation)#记录了输入数据以及隐含层数据
        #print('activation2',np.array(activation).shape)# print('w1',np.array((self.w_[1])).shape)
       
        z = np.dot(self.w_[1], activation.reshape(-1,1)) + self.b_[1]#zl(2)
        print('z',np.array(z).shape)
        zs.append(z) 
        activation = z#self.sigmoid(z)#sigmoid函数计算
        '''隐含层到输出层'''
        activations.append(activation)#至此记录了输入数据、隐含层数据以及输出数据
        
        #开始迭代计算
        delta = self.cost_derivative(activations[-1], y)#* zs[-1]#self.sigmoid_der(zs[-1])
        '''隐含层到输出层'''
        b_new[-1] = delta
        w_new[-1] = np.dot(delta, activations[-2].transpose())
        delta_last = delta.copy()
        z = zs[-2]
        sp = self.phi_der(z, t_new, s_new)
        delta = np.dot(self.w_[-1].transpose(), delta_last) * sp
        b_new[-2] =delta

        w_new[-2] = np.dot(delta, (activations[-3].reshape(-1,1)).transpose())
        sp_t = -.5 * t_new**-1.5 * self.phi((z-s_new) / t_new) - t_new**-2.5 * (z - s_new) * self.phi_der((z - s_new) / t_new)
        sp_s = -t_new**-1.5 * self.phi_der((z-s_new) / t_new)
        # t_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_t # loss函数对小波函数缩放系数的偏导
        # s_new = np.dot(self.w_[-1].transpose(), delta_last)*sp_s # loss函数对小波函数平移系数的偏导
        
        t_new = delta * sp_t # loss函数对小波函数缩放系数的偏导
        s_new = delta * sp_s # loss函数对小波函数平移系数的偏导
        
        return (b_new, w_new, t_new, s_new)
        # 更新权值w，偏移b，缩放因子t，偏移因子s
    def update_mini_batch(self,data, mini_batch, lr):
        b_new = [np.zeros(b.shape) for b in self.b_]
        w_new = [np.zeros(w.shape) for w in self.w_]#a就是输入数据，b就是验证数据
        a, b = mini_batch,data
        n = np.float(mini_batch.size)
        x, y = np.array(a).reshape(-1, 1), np.array(b).reshape(-1, 1)
        delta_b_new, delta_w_new, t_new, s_new = self.backprop(x, y)
        #print('delta_b_new',(np.array(delta_b_new))[0].shape)
        #print('delta_w_new',(np.array(delta_w_new))[0].shape)
        b_new = [nb + dnb for nb, dnb in zip(b_new, delta_b_new)]
        w_new = [nw + dnw for nw, dnw in zip(w_new, delta_w_new)]
           
        self.w_ = [w - lr * nw for w, nw in zip(self.w_, w_new)]
        self.b_ = [b - lr * nb for b, nb in zip(self.b_, b_new)]
        self.t_ = self.t_ - lr * t_new
        self.s_ = self.s_ - lr * s_new
        #print('权值变化',self.w_[1][0]) #,np.array(self.w_[0]).shape,np.array(self.w_[1]).shape)
    
    # training_data是训练数据(x, y), epochs是训练次数, mini_batch_size是每次训练样本数, lr是learning rate，step是展示的迭代间隔
    @timer
    def SGD(self, training_data, epochs=50, mini_batch_size=50, lr=.1, step=10):
        assert type(step) == int, 'Step must be a integer.'
        n = training_data.shape[0]#数据个数
        juli=100
        i=0
        for j in range(epochs):
            ss = training_data
            i+=1
            print('第%d次循环'%(i))
            for k in range(0, n-mini_batch_size, mini_batch_size):
                j=1
                while juli>0.001 and j<1000:
                    np.random.shuffle(ss)#np.random.shuffle(x) 现场修改序列，改变自身内容。（类似洗牌，打乱顺序
                    mini_batches =ss[k:k + mini_batch_size]
                    #print('mini_batches' ,np.array(mini_batches).shape,mini_batches[0] )
                    cs0=ss[k+mini_batch_size:k+mini_batch_size+ 10] #此处对测试进行修改
                    #print('cs0' ,cs0 )
                    #print('cs0' ,np.array(cs0).shape,cs0[0] )
                    #print(np.array(cs0[0]).reshape(-1,1))
                    mini_batch=mini_batches
                    cs=cs0
                    self.update_mini_batch(cs, mini_batch, lr)
                    aa=self.feedforward(mini_batch)
                    print('输出结果',aa)
                    juli=self.oushi(aa,cs)
                    print('误差大小%d'%juli)
                    j+=1
                    print('内循环次数%d'%j)
        
        #print('第%d次循环',%i)
                
            '''
            print(self.oushi(aa,cs))
            d1=self.oushi(aa,cs)
            accur = self.evaluate(training_data) * 100
            mse_loss = self.mse_loss(training_data)
            if (j + 1) % step == 0 or j == 0:
                print("Epoch {0}, mse_loss: {1:.4f}, accury on the training set :{2:.2f}{3}".format(j+1, mse_loss, accur, '%'))
            # print("Epoch {0}: {1} / {2}".format(j, self.evaluate(training_data), n))
            '''
# 计算正确率
    def evaluate(self, data):
        x_t, x_label = data
        test_results = [(np.argmax(self.feedforward(x)), y) for (x, y) in zip(list(x_t), list(x_label))]
        acc = sum(int(x == y) for (x, y) in test_results) / x_t.shape[0]
        return acc
    
    # mse_loss的导数
    def cost_derivative(self, output_activations, y):
        return (output_activations - y)
     # mse_loss
    def mse_loss(self, training_data):
        x_t,x_label = training_data
        test_results = [.5 * norm(self.feedforward(x).flatten() - self.one_hot(y, num_classes=10))**2
                        for (x, y) in zip(list(x_t), list(x_label))]
        return np.array(test_results).mean()
    
    # 预测
    def predict(self, data):
        data = data.reshape(-1, self.sizes_[0])
        value = np.array([np.argmax(net.feedforward(x)) for x in data], dtype='uint8')
        return value
    
    # 保存训练模型
    def save(self):
        pass  # 把_w和_b保存到文件(pickle)  
    
    def load(self):
        pass            

In [8]:
import pandas as pd
a0=pd.read_csv(r'C:\Users\stu\Desktop\xlab_时许数据\CPU.csv').values
a0[:,1].size;a1=a0[:,1];a1=a1.tolist();type(a1)
x=a1[:500]
x2=a1[500:510]
training_data=np.array(x)
print(training_data.size)
type(training_data)
#sizes=[len(x), 128, 10]
#sizes_ = sizes
#num_layers_ = len(sizes)
#print(num_layers_)# 层数
#if num_layers_ > 3:
    #print('ERROR!')
#num_nuerals_ = sizes[1]#隐含层个数
#print(num_nuerals_)
#w_ = [np.random.randn(y, x) for x, y in zip(sizes[:-1], sizes[1:])]  
#b_ = [np.random.randn(y, 1) for y in sizes[1:]]
#t_ = np.random.randint(2, 15, (num_nuerals_, 1))
#s_ = 2 * np.random.randn(num_nuerals_, 1)

500


numpy.ndarray

In [10]:
net = WaveletNeuralNet([50,128,10])
bf=net.feedforward(np.array(x)[0:50])
print(bf)
#print(bf)
x2=a1[500:510]
xx=pd.DataFrame(x2)
print(xx.values)
xx2=xx.values
print('xx2',xx2)
[n1,n2,n3,n4]=net.backprop(np.array(x[0:50]),xx2)
np.array(n2)[1].shape
n2

[[-0.70267289]
 [-0.07643722]
 [-0.32553032]
 [-2.20032755]
 [-0.14839752]
 [ 0.015463  ]
 [ 1.14991599]
 [ 1.60360309]
 [-0.329563  ]
 [-0.10401748]]
[[58.06666667]
 [57.68333333]
 [57.28475717]
 [58.1       ]
 [58.03333333]
 [57.6       ]
 [57.5       ]
 [57.91666667]
 [58.45      ]
 [58.41528051]]
xx2 [[58.06666667]
 [57.68333333]
 [57.28475717]
 [58.1       ]
 [58.03333333]
 [57.6       ]
 [57.5       ]
 [57.91666667]
 [58.45      ]
 [58.41528051]]
z (10, 1)


[array([[ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00],
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        ...,
        [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00, ...,
          0.00000000e+00,  0.00000000e+00,  0.00000000e+00],
        [ 2.76588728e-13,  2.78973205e-13,  2.74263715e-13, ...,
          2.77292667e-13,  2.76741419e-13,  2.78255845e-13],
        [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00, ...,
         -0.00000000e+00, -0.00000000e+00, -0.00000000e+00]]),
 array([[-0.00000000e+00,  0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  1.53136932e-15, -0.00000000e+00],
        [-0.00000000e+00,  0.00000000e+00, -0.00000000e+00, ...,
          0.00000000e+00,  1.50506270e

In [ ]:
net = WaveletNeuralNet([50,500,10])
net.SGD(training_data, epochs=2, mini_batch_size=50, lr=.1, step=20)

In [18]:
net.feedforward(np.array(x)[0:50])

array([[58.04749659],
       [59.10030716],
       [57.86762371],
       [57.98580398],
       [58.1100422 ],
       [57.87074189],
       [58.01195216],
       [57.85867528],
       [57.99325519],
       [58.00245571]])

In [21]:
cc1=a1[50:60]

In [20]:
cc=[57.92731745,57.92206549,57.94790921,57.94092723,58.09708394,57.80595973,57.8869547,57.90202747,57.74416407,58.08103361]

In [22]:
net.oushi(cc1,cc)

1.6815586036409542

In [40]:
import numpy as np

def sigmoid(x):
    return 1 / (1 + np.exp(-x))
def sigmoidDerivationx(y):
    return y * (1 - y)


if __name__ == '__main__':
    # 初始化一些参数
    alpha = 0.5
    numIter = 10#迭代次数
    w1 = [[0.15, 0.20], [0.25, 0.30]]  # Weight of input layer
    w2 = [[0.40, 0.45], [0.50, 0.55]]
    # print(np.array(w2).T)
    b1 = 0.35
    b2 = 0.60
    x = [0.05, 0.10]
    y = [0.01, 0.99]
    # 前向传播
    z1 = np.dot(w1, x) + b1     # dot函数是常规的矩阵相乘
    a1 = sigmoid(z1)

    z2 = np.dot(w2, a1) + b2
    a2 = sigmoid(z2)
    for n in range(numIter):
        # 反向传播 使用代价函数为C=1 / (2n) * sum[(y-a2)^2]
        # 分为两次
        # 一次是最后一层对前面一层的错误

        delta2 = np.multiply(-(y-a2), np.multiply(a2, 1-a2))
        # for i in range(len(w2)):
        #     print(w2[i] - alpha * delta2[i] * a1)
        #计算非最后一层的错误
        # print(delta2)
        delta1 = np.multiply(np.dot(np.array(w2).T, delta2), np.multiply(a1, 1-a1))
        # print(delta1)
        # for i in range(len(w1)):
            # print(w1[i] - alpha * delta1[i] * np.array(x))
        #更新权重
        for i in range(len(w2)):
            w2[i] = w2[i] - alpha * delta2[i] * a1
        for i in range(len(w1)):
            w1[i] = w1[i] - alpha * delta1[i] * np.array(x)
        #继续前向传播，算出误差值
        z1 = np.dot(w1, x) + b1
        a1 = sigmoid(z1)
        z2 = np.dot(w2, a1) + b2
        a2 = sigmoid(z2)
        print(str(n) + " result:" + str(a2[0]) + ", result:" +str(a2[1]))
        # print(str(n) + "  error1:" + str(y[0] - a2[0]) + ", error2:" +str(y[1] - a2[1]))

0 result:0.7420881111907824, result:0.7752849682944595
1 result:0.7324768705997207, result:0.7775819271030018
2 result:0.722538848562922, result:0.7798215827742319
3 result:0.7122848772449228, result:0.7820060734063023
4 result:0.7017292761322197, result:0.7841374389249309
5 result:0.6908899293110883, result:0.7862176255712456
6 result:0.6797882700461756, result:0.7882484901472318
7 result:0.6684491619864679, result:0.7902318040469472
8 result:0.6569006710842666, result:0.7921692571023269
9 result:0.6451737282231507, result:0.7940624612716817


In [5]:
aa=1;bb=2
i=0;j=1
while aa<100 and j<20:
    aa+=1
    j+=1

In [6]:
aa

20